In [13]:
%load_ext autoreload
%autoreload 2
import sys
import torch
import pickle
from torch import nn
sys.path.append('..')
# sys.path.append('/system/user/beck/pwbeck/projects/regularization/ml_utilities')
from pathlib import Path
import matplotlib.pyplot as plt
from ml_utilities.torch_models.fc import FC
from erank.utils import load_model_from_epoch, load_directions_matrix_from_task_sweep, load_multiple_dir_matrices_from_sweep
from erank.plot import plot_models_erank
from erank.analyses import create_model_erank_df

gpu_id = 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Erank plots

### Create model dict

In [2]:
sweep_dir = '/system/user/beck/pwbeck/projects/regularization/erank/outputs/sl-sine-10.0.1-sl-randinit-ampl-XXX-ph-XXX-taskidx-XXX_220912_234805'
sweep_output_path = Path(sweep_dir) / 'outputs'
num_vectors=1900

In [3]:
# check for missing best_epoch_files
# for p in sweep_output_path.iterdir():
#     bepf = p / 'best_epoch.txt'
#     if not bepf.exists():
#         print(str(p))

In [4]:
sine_task_distr = {'normal': '*ampl-0.1_5.0-ph-0_3.141592654*', 
                    'wide': '*ampl-0.1_10.0-ph-0_3.141592654*',
                    'narrow': '*ampl-0.1_2.5-ph-0_1.5*',
                    'almost_point': '*ampl-1.0_1.1-ph-0_0.1*'}

In [5]:
len(sine_task_distr)

4

In [6]:
# load a model
# run = next(iter(sweep_output_path.iterdir()))
# init_model = load_model_from_epoch(run, epoch=0, device='cpu')

In [7]:
# load model matrices
# model_dict = load_multiple_dir_matrices_from_sweep(path_to_runs=sweep_output_path,
#                                                    name_run_glob_pattern_dict=sine_task_distr,
#                                                    num_runs=num_vectors,
#                                                    device='cpu')
# with open(f'model_dict_num_vecs{num_vectors}.p', 'wb') as savefile:
#     pickle.dump(model_dict, savefile, protocol=pickle.HIGHEST_PROTOCOL)

### Load model dict

In [15]:
model_dict_path = './../tmp/model_dict_num_vecs1900.p'
model_dict_path = Path(model_dict_path)
init_model_path = './../tmp/init_model.p'
init_model_path = Path(init_model_path)

In [20]:
# load init model
init_model = FC.load(init_model_path, device=gpu_id)
init_model, init_model.device

(FC(
   (fc): Sequential(
     (0): Flatten(start_dim=1, end_dim=-1)
     (1): Linear(in_features=1, out_features=40, bias=True)
     (2): ReLU(inplace=True)
     (3): Linear(in_features=40, out_features=40, bias=True)
     (4): ReLU(inplace=True)
     (5): Linear(in_features=40, out_features=1, bias=True)
   )
 ),
 device(type='cuda', index=0))

In [18]:
# unpickle model dict
with open(model_dict_path, 'rb') as loadfile:
    model_dict = pickle.load(loadfile)

In [19]:
list(model_dict.keys()), type(model_dict)

(['normal#*ampl-0.1_5.0-ph-0_3.141592654*',
  'wide#*ampl-0.1_10.0-ph-0_3.141592654*',
  'narrow#*ampl-0.1_2.5-ph-0_1.5*',
  'almost_point#*ampl-1.0_1.1-ph-0_0.1*'],
 dict)

### Generate erank plot

In [21]:
erank_ = create_model_erank_df(models=model_dict, random_init_model=init_model, device=gpu_id)

Calculate eranks:  75%|███████▌  | 144/191 [05:42<06:14,  7.97s/it]

In [11]:
fig, ax = plot_models_erank(models=model_dict, random_init_model=init_model, device='cuda')

NameError: name 'init_model' is not defined

In [ ]:
fig.savefig(f'sinus_erank_varying_task_distribution_num_vecs{num_vectors}.pdf', bbox_inches='tight')

In [ ]:
fig.savefig(f'sinus_erank_varying_task_distribution_num_vecs{num_vectors}.png', dpi=300, bbox_inches='tight')